# Imports

In [ ]:
import sys
sys.path.append('/home/rcendre/classification')
import os
import sys
import itertools
import webbrowser
from pathlib import Path
import matplotlib.pyplot as plt
from numpy import array, logspace
from scipy.stats import randint as randint
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import davies_bouldin_score
from toolbox.classification.common import Data, Folds, IO, Tools
from toolbox.classification.parameters import Dermatology, Settings
from toolbox.IO import dermatology
from toolbox.models.builtin import Applications
from toolbox.models.models import DecisionVotingClassifier, ScoreVotingClassifier
from toolbox.transforms.common import PredictorTransform
from toolbox.transforms.labels import OrderedEncoder
from toolbox.transforms.images import DistributionImageTransform, DWTImageTransform, FourierImageTransform, HaralickImageTransform, SpatialImageTransform
from toolbox.views.common import Views, ViewsTools
from toolbox.views.images import ImagesViews
from IPython.display import display
import warnings
warnings.filterwarnings('ignore') 

# Parameters

In [ ]:
# Advanced parameters
data_type = 'Full'
extractor = 'Spatial'
validation = 4
settings = Settings.get_default_dermatology()

# Inputs

In [ ]:
inputs = IO.load(f'Features.pickle')

In [ ]:
inputs = inputs[(inputs['Binary_Diagnosis'] == 'Benign') |(inputs['Binary_Diagnosis'] == 'Malignant')].reset_index(drop=True)

# Low level predictions

In [ ]:
low_file = f'Low_Calibration_Spatial.pickle'

In [ ]:
low_folds = [([1, 2], [3]),
             ([2, 3], [4]),
             ([3, 4], [1]),
             ([4, 1], [2])]

In [ ]:
# Add scaling step
supervised = Pipeline([ ('scale', MinMaxScaler()), ('clf', SVC(kernel='linear', class_weight='balanced', probability=True))])
supervised_params = {'clf__C': logspace(-2, 3, 6).tolist()}

In [ ]:
# Transform standard inputs
Tools.fit_predict(inputs, {'datum': extractor, 'label_encode': 'LesionEncode'}, 
                  supervised, 'SupervisedIsotonic', folds=low_folds, distribution=supervised_params, calibrate='isotonic')

Tools.fit_predict(inputs, {'datum': extractor, 'label_encode': 'LesionEncode'}, 
                  supervised, 'SupervisedSigmoid', folds=low_folds, distribution=supervised_params, calibrate='sigmoid')

In [ ]:
all_image = [True] * len(inputs.index)
single_image = inputs['ID_Image'] == '0M'
Data.build_bags(inputs, single_image, 'ID_Lesion', all_image, 'ID_Lesion', f'SupervisedIsotonic_{Tools.PREDICTION}')
Data.build_bags(inputs, single_image, 'ID_Lesion', all_image, 'ID_Lesion', f'SupervisedIsotonic_{Tools.PROBABILITY}')

Data.build_bags(inputs, single_image, 'ID_Lesion', all_image, 'ID_Lesion', f'SupervisedSigmoid_{Tools.PREDICTION}')
Data.build_bags(inputs, single_image, 'ID_Lesion', all_image, 'ID_Lesion', f'SupervisedSigmoid_{Tools.PROBABILITY}')
inputs = inputs[single_image].reset_index()

In [ ]:
# Save
IO.save(inputs, low_file)

# High level predictions

In [ ]:
inputs = IO.load(low_file)

In [ ]:
high_file = f'High_Calibration_Spatial.pickle'

In [ ]:
high_folds = [([4], [1]),
             ([1], [2]),
             ([2], [3]),
             ([3], [4])]

In [ ]:
# Isotonic
# Decisions
Tools.evaluate(inputs, {'datum': f'SupervisedIsotonic_{Tools.PREDICTION}', 'label_encode': 'LesionEncode'}, DecisionVotingClassifier(mode='at_least_one'), 'D_ALOIsotonic', folds=high_folds)
Tools.evaluate(inputs, {'datum': f'SupervisedIsotonic_{Tools.PREDICTION}', 'label_encode': 'LesionEncode'}, DecisionVotingClassifier(mode='dynamic_thresh'), 'D_DYNIsotonic', folds=high_folds)

# Score
Tools.evaluate(inputs, {'datum': f'SupervisedIsotonic_{Tools.PROBABILITY}', 'label_encode': 'LesionEncode'}, ScoreVotingClassifier(low='max', high='dynamic'), 'S_MaxDIsotonic', folds=high_folds)
Tools.evaluate(inputs, {'datum': f'SupervisedIsotonic_{Tools.PROBABILITY}', 'label_encode': 'LesionEncode'}, ScoreVotingClassifier(low='mean', high='dynamic'), 'S_MeanDIsotonic', folds=high_folds)

# Sigmoid
# Decisions
Tools.evaluate(inputs, {'datum': f'SupervisedSigmoid_{Tools.PREDICTION}', 'label_encode': 'LesionEncode'}, DecisionVotingClassifier(mode='at_least_one'), 'D_ALOSigmoid', folds=high_folds)
Tools.evaluate(inputs, {'datum': f'SupervisedSigmoid_{Tools.PREDICTION}', 'label_encode': 'LesionEncode'}, DecisionVotingClassifier(mode='dynamic_thresh'), 'D_DYNSigmoid', folds=high_folds)

# Score
Tools.evaluate(inputs, {'datum': f'SupervisedSigmoid_{Tools.PROBABILITY}', 'label_encode': 'LesionEncode'}, ScoreVotingClassifier(low='max', high='dynamic'), 'S_MaxDSigmoid', folds=high_folds)
Tools.evaluate(inputs, {'datum': f'SupervisedSigmoid_{Tools.PROBABILITY}', 'label_encode': 'LesionEncode'}, ScoreVotingClassifier(low='mean', high='dynamic'), 'S_MeanDSigmoid', folds=high_folds)

# High Level
IO.save(inputs, high_file)

# Analysis

## Scores and ROC

In [ ]:
from IPython.display import HTML
from IPython.display import display

inputs = IO.load(high_file)

diagnosis_encoder = OrderedEncoder().fit(['Benign', 'Malignant'])
# ROC Curve
ViewsTools.plot_size((8,8))
high_models = ['D_ALOIsotonic', 'D_DYNIsotonic', 'S_MaxDIsotonic', 'S_MeanDIsotonic',
               'D_ALOSigmoid', 'D_DYNSigmoid', 'S_MaxDSigmoid', 'S_MeanDSigmoid']
for name in high_models: 
    #Label
    display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'LesionEncode', 'eval': name}, diagnosis_encoder)],
                                                                        title=[f'Test - {name}'])))